# Madison tech meetup "yes" RSVPs vs. time

In [ ]:
import pandas as pd
import numpy as np
from textwrap import wrap
import matplotlib.pyplot as plt

from madison_meetups import get_madison_tech_groups, get_all_group_events

%matplotlib inline

Load information about each tech meetup group in Madison, WI. 

<div class="alert alert-block alert-info">
    _NOTE_: The meetup api will write lots of things to `sys.stdout` and `sys.sterr`. The built-in IPython `%%capture` magic command will supress this (for more info see http://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-capture)
</div>

In [ ]:
%%capture
madison_tech_groups = get_madison_tech_groups()

In [ ]:
madison_tech_groups.head()

Relevant data for each meetup group

In [ ]:
madison_tech_groups.columns

Load Madison tech meetup event data (this may take several seconds)

In [ ]:
%%capture
group_events = get_all_group_events(madison_tech_groups)

Each row in `group_events` represents a meetup event

In [ ]:
group_events.head()

In [ ]:
group_events.columns

Convert the `time` column in `group_events` to `datetime` objects

In [ ]:
group_events['event_time'] = group_events.time.apply(pd.to_datetime, unit='ms')
group_events['event_time'].head()

Get groups with largest number of members 

In [ ]:
num_top = 5
top_names = list(madison_tech_groups.loc[madison_tech_groups.members.nlargest(num_top).index, 'name'])
if 'Madpy' not in top_names:
    top_names.append('Madpy')
top_names

Plot the number of "yes" RSVPs vs. time for MadPy + top Madison tech meetups

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for group_name, group in group_events.groupby('group_name'):
    if group_name not in top_names:
        continue
    else:
        time_series = pd.Series(group['yes_rsvp_count'].values, index=group['event_time'])
        
        if group_name == 'Madpy':
            color = 'k'
            lw = 2
        else:
            color = None
            lw = 1
            
        # Don't want really long names in single line of legend
        label = '\n'.join(wrap(group_name, 20))
        time_series.plot(label=label,
                         lw=lw,
                         color=color,
                         xlim=('2017', pd.Timestamp.today()),
                         ax=ax)
ax.set_ylim(0)
ax.set_xlabel('')
ax.set_ylabel('Number of "yes" RSVPs')

ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.grid(axis='y', ls=':')
plt.tight_layout()
plt.show()